## 简单的命令行执行，获取命令行输出

In [1]:
import subprocess

completed = subprocess.run(['dir', '/d'], shell=True, encoding='gbk')
print('returncode:', completed.returncode)

# safer version
completed = subprocess.run(['cmd', '/c', 'dir /d'], capture_output=True, text=True, encoding='gbk')
print(completed.stdout)
print('returncode:', completed.returncode)

completed = subprocess.run(['cmd', '/c', 'echo %USERPROFILE%'], shell=True)
print('returncode:', completed.returncode)


completed = subprocess.run(['cmd', '/c', 'dir', '/d'], capture_output=True, text=True, encoding='gbk')
print('returncode:', completed.returncode)
print('stdout:', completed.stdout)
print('stderr:', completed.stderr)



returncode: 0
 驱动器 D 中的卷是 Data
 卷的序列号是 D62E-479E

 d:\VSCODE\PythonPractice 的目录

[.]                      multi_threads.ipynb      
[..]                     README.md                
fun.py                   subprocess_practice.py   
               4 个文件         35,032 字节
               2 个目录 66,418,671,616 可用字节

returncode: 0
returncode: 0
returncode: 0
stdout:  驱动器 D 中的卷是 Data
 卷的序列号是 D62E-479E

 d:\VSCODE\PythonPractice 的目录

[.]                      multi_threads.ipynb      
[..]                     README.md                
fun.py                   subprocess_practice.py   
               4 个文件         35,032 字节
               2 个目录 66,418,671,616 可用字节

stderr: 


In [2]:
# 简单的命令行执行
completed = subprocess.run(['dir', '/d'], shell=True)
print('returncode:', completed.returncode)

# 获取命令行输出
output = subprocess.check_output(['dir'], shell=True, text=True, encoding='gbk')
print(output)


returncode: 0
 驱动器 D 中的卷是 Data
 卷的序列号是 D62E-479E

 d:\VSCODE\PythonPractice 的目录

2023-03-08  20:10    <DIR>          .
2023-03-08  20:10    <DIR>          ..
2023-03-08  19:51             3,826 fun.py
2023-03-08  20:10            21,404 multi_threads.ipynb
2023-03-07  08:34                81 README.md
2023-03-08  20:10             9,721 subprocess_practice.py
               4 个文件         35,032 字节
               2 个目录 66,418,671,616 可用字节



### subprocess.run(stdout=, stderr=)

In [3]:
import subprocess

try:
    completed = subprocess.run(
        'dir /d', shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='gbk')
except subprocess.CalledProcessError as err:
    print('ERROR:', err)
    print('stderr:', err.stderr)

else:
    print('returncode:', completed.returncode)
    print('stdout:', completed.stdout)
    print('stderr:', completed.stderr)

returncode: 0
stdout:  驱动器 D 中的卷是 Data
 卷的序列号是 D62E-479E

 d:\VSCODE\PythonPractice 的目录

[.]                      multi_threads.ipynb      
[..]                     README.md                
fun.py                   subprocess_practice.py   
               4 个文件         35,032 字节
               2 个目录 66,418,671,616 可用字节

stderr: 


### subprocess.run(timeout=)

In [4]:
import subprocess

try:
    completed = subprocess.run(['ping', 'www.baidu.com', '-t'], timeout=5)
except subprocess.TimeoutExpired:
    print('Ping timed out after 5 seconds.')

Ping timed out after 5 seconds.


### threading.Thread

In [5]:
import threading

class BankAccount:
    """银行账户类"""

    def __init__(self, balance=0):
        """初始化账户余额"""
        self.balance = balance

    def withdraw(self, amount):
        """取款操作"""
        if self.balance >= amount:
            self.balance -= amount
            print(f"Withdraw {amount} succeeded")
        else:
            print(f"Withdraw {amount} failed")

# 创建一个银行账户实例
account = BankAccount(balance=100)

# 创建多个线程进行取款操作
threads = []
for i in range(10):
    t = threading.Thread(target=account.withdraw, args=(10,))
    threads.append(t)

# 启动多个线程
for t in threads:
    t.start()

# 等待所有线程执行完毕
for t in threads:
    t.join()

# 输出最终余额
print(f"Final balance: {account.balance}")

Withdraw 10 succeededWithdraw 10 succeeded
Withdraw 10 succeeded

Withdraw 10 succeeded
Withdraw 10 succeeded
Withdraw 10 succeeded
Withdraw 10 succeeded
Withdraw 10 succeeded
Withdraw 10 succeeded
Withdraw 10 succeeded
Final balance: 0


### threading.lock()锁机制：数据竞争

In [6]:
import threading
import time
balance = 0
lock = threading.Lock()

def deposit(n):
    global balance
    for i in range(1000000):
        lock.acquire()      # 获取锁
        balance += n
        lock.release()      # 释放锁

def withdraw(n):
    global balance
    for i in range(1000000):
        lock.acquire()      # 获取锁
        balance -= n
        lock.release()      # 释放锁

if __name__ == '__main__':
    start_time = time.time()
    t1 = threading.Thread(target=deposit, args=(1,))
    t2 = threading.Thread(target=withdraw, args=(1,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print('balance:', balance)
    print('timecost:',time.time()-start_time)

balance: 0
timecost: 0.8091800212860107


### with self.lock:用类改写，with自动释放锁

In [7]:
import threading

class BankAccount:
    def __init__(self, balance):
        self.balance = balance
        self.lock = threading.Lock()

    def deposit(self, amount):
        with self.lock:
            self.balance += amount
            print(f"Deposited {amount}, balance is now {self.balance}")

    def withdraw(self, amount):
        with self.lock:
            if self.balance >= amount:
                self.balance -= amount
                print(f"Withdrew {amount}, balance is now {self.balance}")
            else:
                print(f"Insufficient balance, cannot withdraw {amount}")

def run_threads(account):
    for _ in range(2):
        account.deposit(1)
        account.withdraw(1)

if __name__ == '__main__':
    account = BankAccount(0)
    threads = [threading.Thread(target=run_threads, args=(account,)) for _ in range(2)]

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print(f"Final balance: {account.balance}")

Deposited 1, balance is now 1
Withdrew 1, balance is now 0
Deposited 1, balance is now 1
Withdrew 1, balance is now 0
Deposited 1, balance is now 1
Withdrew 1, balance is now 0
Deposited 1, balance is now 1
Withdrew 1, balance is now 0
Final balance: 0


### 队列-生产者消费者模型

In [8]:
import threading
import time
import queue

# 定义队列对象
q = queue.Queue()

# 定义生产者线程
class ProducerThread(threading.Thread):
    def __init__(self, exit_event):
        super().__init__()
        self.exit_event = exit_event
    def run(self):
        global q
        count = 0
        while not self.exit_event.is_set():
            if q.qsize() < 5:
                for i in range(5):
                    count += 1
                    q.put('product-' + str(count))
                print('produced 5 products')
            time.sleep(2)

# 定义消费者线程
class ConsumerThread(threading.Thread):
    def __init__(self, exit_event):
        super().__init__()
        self.exit_event = exit_event
    def run(self):
        global q
        while not self.exit_event.is_set():
            if q.qsize() > 0:
                product = q.get()
                print('consumed product:', product)
            time.sleep(1)
stop_event = threading.Event()
# 创建生产者线程和消费者线程，并启动它们
producer_thread = ProducerThread(stop_event)
consumer_thread = ConsumerThread(stop_event)
producer_thread.start()
consumer_thread.start()
time.sleep(3)
stop_event.set()
producer_thread.join()
consumer_thread.join()

produced 5 productsconsumed product:
 product-1
consumed product: product-2
produced 5 products
consumed product: product-3


### threading.Timer(interval, function) 多少秒后执行function

In [9]:
import threading

def my_timer():
    print("定时器执行了！")

timer = threading.Timer(10.0, my_timer)
timer.start()

# 等待3秒后取消定时器的执行
event = threading.Event()
event.wait(3.0)
if not event.is_set():
    timer.cancel()
    print("定时器已取消！")

定时器已取消！


### logging：输出日志

In [10]:
import logging

logging.basicConfig(level=logging.DEBUG)

def foo():
    logging.debug("Debug information")
foo()

DEBUG:root:Debug information


### threading.Thread(daemon=True)

In [11]:
import threading
import time

def print_time():
    i = 0   # something wrong with vscode's ipynb,
            # this won't stop when main thread ends,
            # so I have to add this.
    while True and i < 5:
        print("Current time is {}".format(time.time()))
        time.sleep(1)
        i += 1

if __name__ == '__main__':
    t = threading.Thread(target=print_time, daemon=True)
    t.start()

    time.sleep(3)
    print("Main thread ends")

Current time is 1678277473.6407301
Current time is 1678277474.6513364
Current time is 1678277475.6639278
Main thread ends


### threading.Event()

In [12]:
import threading
import time

# 创建 Event 对象
event = threading.Event()

def wait_for_event():
    print("Thread {} is waiting for event...".format(threading.current_thread().name))
    event.wait()
    print("Thread {} received the event!".format(threading.current_thread().name))

def set_event():
    print("Thread {} is sleeping for 3 seconds...".format(threading.current_thread().name))
    time.sleep(2)
    event.set()
    print("Thread {} set the event!".format(threading.current_thread().name))

# 创建两个线程
t1 = threading.Thread(target=wait_for_event)
t2 = threading.Thread(target=set_event)

# 启动两个线程
t1.start()
t2.start()

# 等待两个线程结束
t1.join()
t2.join()

Current time is 1678277476.677808
Thread Thread-27 is waiting for event...
Thread Thread-28 is sleeping for 3 seconds...
Current time is 1678277477.690457
Thread Thread-28 set the event!Thread Thread-27 received the event!



In [13]:
import queue
import threading
import time
import random

class Producer(threading.Thread):
    def __init__(self, queue, event):
        super().__init__()
        self.queue = queue
        self.event = event

    def run(self):
        while not self.event.is_set():
            data = random.randint(0, 100)
            self.queue.put(data)
            print(f"Produced {data}")
            time.sleep(1)

class Consumer(threading.Thread):
    def __init__(self, queue, event):
        super().__init__()
        self.queue = queue
        self.event = event

    def run(self):
        while not self.event.is_set() or not self.queue.empty():
            try:
                data = self.queue.get(timeout=5)
                print(f"Consumed {data}")
            except queue.Empty:
                # something wrong with 'queue.Empty' 
                print("Queue is empty")

if __name__ == "__main__":
    queue = queue.Queue()
    event = threading.Event()

    producer = Producer(queue, event)
    consumer = Consumer(queue, event)

    producer.start()
    consumer.start()

    time.sleep(3)
    event.set()
    print("Event is set")

Produced 47Consumed 47

Produced 82Consumed 82

Produced 36Consumed 36

Event is set


### threading.Condition()

In [14]:
import threading

class Printer:
    def __init__(self):
        self.lock = threading.Lock()
        self.cond = threading.Condition(self.lock)
        self.num = 0

    def odd(self):
        with self.cond:
            while self.num < 10:
                if self.num % 2 == 0:
                    self.cond.wait()
                else:
                    print(self.num)
                    self.num += 1
                    self.cond.notify()

    def even(self):
        with self.cond:
            while self.num < 10:
                if self.num % 2 != 0:
                    self.cond.wait()
                else:
                    print(self.num)
                    self.num += 1
                    self.cond.notify()

p = Printer()
t1 = threading.Thread(target=p.odd)
t2 = threading.Thread(target=p.even)
t1.start()
t2.start()
t1.join()
t2.join()

0
1
2
3
4
5
6
7
8
9


### threading.Barrier(parties, action, timeout)

In [15]:
import threading

def worker(barrier):
    print(f"Worker thread {threading.current_thread().name} is waiting at the barrier...")
    barrier.wait()
    print(f"Worker thread {threading.current_thread().name} passed the barrier!")

def my_action():
    print("All threads have arrived at the barrier!")

if __name__ == "__main__":
    num_threads = 5
    
    barrier = threading.Barrier(num_threads, action=my_action)

    for i in range(num_threads):
        t = threading.Thread(target=worker, args=(barrier,), name=f"Thread-{i}")
        t.start()

Worker thread Thread-0 is waiting at the barrier...
Worker thread Thread-1 is waiting at the barrier...
Worker thread Thread-2 is waiting at the barrier...
Worker thread Thread-3 is waiting at the barrier...
Worker thread Thread-4 is waiting at the barrier...
All threads have arrived at the barrier!
Worker thread Thread-3 passed the barrier!
Worker thread Thread-0 passed the barrier!Worker thread Thread-1 passed the barrier!Worker thread Thread-4 passed the barrier!

Worker thread Thread-2 passed the barrier!



### threading.Event.is_set()

In [16]:
import threading
import time

def counter(stop_event):
    count = 0
    while not stop_event.is_set():
        # print("stop_event hasn't been set.")
        count += 1
        print(f"Count: {count}")
        time.sleep(0.5)

def main():
    stop_event = threading.Event()
    threads = []
    for i in range(2):
        t = threading.Thread(target=counter, args=(stop_event,))
        t.start()
        threads.append(t)
    
    time.sleep(3)
    print("stop_event has been set.")
    stop_event.set()

    for t in threads:
        t.join()

if __name__ == '__main__':
    main()

Count: 1Count: 1

Count: 2Count: 2

Count: 3Count: 3

Count: 4
Count: 4
Count: 5Count: 5

Count: 6
Count: 6
stop_event has been set.


Exception in thread Thread-30:
Traceback (most recent call last):
  File "C:\Users\Matrix\AppData\Local\Temp\ipykernel_24100\2297608760.py", line 28, in run
  File "d:\Python\anaconda3\lib\queue.py", line 179, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Python\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Matrix\AppData\Local\Temp\ipykernel_24100\2297608760.py", line 30, in run
AttributeError: 'Queue' object has no attribute 'Empty'
